# SQL

## Conexión a PostgreSQL usando SQLAlchemy

In [1]:
import os
import pandas as pd
from typing import Union
from sqlalchemy import create_engine, text

# Reemplaza USER y PASSWORD del .env
engine = create_engine("sqlite:///data/sql-murder-mystery.db")

# Probar la conexión
try:
    connection = engine.connect()
    print("Conexión exitosa")
    connection.close()
except Exception as e:
    print(f"Error al conectar: {e}")


def ejecutar_query(query: str) -> Union[pd.DataFrame, str]:
    """
    Ejecuta una consulta SQL en la base de datos y devuelve un DataFrame.

    Parámetros:
    - query (str): La consulta SQL a ejecutar.

    Retorna:
    - Union[pd.DataFrame, pl.DataFrame, str]: Retorna un DataFrame en pandas o polars dependiendo del parámetro 'devolver_polars',
      o un mensaje de éxito/error si la operación no es un SELECT.
    """
    
    with engine.connect() as connection:
        try:
            result = connection.execute(text(query))
            
            # Determinar si es una consulta SELECT para devolver datos
            if result.returns_rows:
                rows = result.fetchall()
                columns = result.keys()

                # Convertir filas en un diccionario de listas para crear correctamente el DataFrame en polars
                data_dict = {col: [row[idx] for row in rows] for idx, col in enumerate(columns)}

                # Verificación de coincidencia de columnas y filas
                return pd.DataFrame(data_dict)
            else:
                connection.commit()
                return f"Query ejecutada correctamente: {result.rowcount} filas afectadas."
        except Exception as e:
            return f"Error al ejecutar la query: {e}"


Conexión exitosa


In [9]:
pd.set_option('display.max_colwidth',None)
#pd.set_option('display.max_columns', None) # Alarga la columna

query= """SELECT * 
          FROM crime_scene_report"""


ejecutar_query(query=query)

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his head...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to the knee!,SQL City
...,...,...,...,...
1223,20180430,bribery,\n,Garden Grove
1224,20180430,fraud,‘Why not?’ said the March Hare.\n,Houma
1225,20180430,assault,\n,Fontana
1226,20180501,assault,"be NO mistake about it: it was neither more nor less than a pig, and she\n",Trenton


In [ ]:
pd.set_option('display.max_colwidth',None)
#pd.set_option('display.max_columns', None) # Alarga la columna

query= """SELECT * 
          FROM crime_scene_report
          WHERE type = 'murder'
          AND city = 'SQL City'
          AND date = '20180115'"""

ejecutar_query(query=query)

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [11]:
query ='SELECT * FROM person WHERE address_street_name = "Northwestern Dr" OR address_street_name = "Franklin Ave"'

ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,10010,Muoi Cary,385336,741,Northwestern Dr,828638512
1,12207,Wilmer Wolever,509484,139,Franklin Ave,636825374
2,12711,Norman Apolito,667757,599,Northwestern Dr,778264744
3,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949
4,15171,Weldon Penso,336999,311,Northwestern Dr,131379495
...,...,...,...,...,...,...
90,96595,Coretta Cubie,303645,3631,Northwestern Dr,378403829
91,97913,Cameron Dilick,971988,2954,Franklin Ave,665147939
92,98593,Jonah Toner,578982,2947,Northwestern Dr,745853548
93,98744,Jordan Myntti,256116,3104,Franklin Ave,876030104


In [16]:
query= 'SELECT * FROM person WHERE address_street_name = "Northwestern Dr" AND address_number > 4918 '

ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [37]:
query= 'SELECT * FROM person WHERE address_street_name = "Franklin Ave"'

ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,12207,Wilmer Wolever,509484,139,Franklin Ave,636825374
1,16371,Annabel Miller,490173,103,Franklin Ave,318771143
2,17683,Johnnie Schee,968887,1277,Franklin Ave,815977821
3,18651,Carleen Etoll,356746,22,Franklin Ave,193369255
4,22636,Zachary Ybarbo,768359,785,Franklin Ave,285346605
5,24737,Gema Nantz,273410,3968,Franklin Ave,180545802
6,30654,Clarita Rickels,418084,2254,Franklin Ave,714941023
7,32264,Shelby Dezeeuw,735415,1391,Franklin Ave,143197463
8,33793,Amado Mattan,161915,99,Franklin Ave,125205748
9,34592,Cordell Lindamood,592762,3657,Franklin Ave,509890333


In [36]:
query = """SELECT * FROM person 
WHERE address_street_name = 'Franklin Ave' 
AND name = 'Annabel Miller'"""

ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [ ]:
# Entrevista a Annabel
query= 'SELECT * FROM interview WHERE person_id = "16371" '

ejecutar_query(query=query)

,person_id,transcript
0,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [ ]:
# Entrevista a Morty
query= 'SELECT * FROM interview WHERE person_id = "14887"'

ejecutar_query(query=query)

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."


In [ ]:
# The membership number on the bag started with "48Z". Only gold members have those bags.
query= 'SELECT * FROM get_fit_now_member WHERE membership_status = "gold" AND id LIKE "48Z%"'

ejecutar_query(query=query)

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


In [ ]:
# Joe Germuska
query= 'SELECT * FROM get_fit_now_check_in WHERE membership_id = "48Z7A"'

ejecutar_query(query=query)

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730


In [ ]:
# Jeremy Bowers
query= 'SELECT * FROM get_fit_now_check_in WHERE membership_id = "48Z55"'

ejecutar_query(query=query)

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z55,20180109,1530,1700


In [62]:
query= 'SELECT * FROM person WHERE name = "Jeremy Bowers" OR name ="Joe Germuska"'

ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,28819,Joe Germuska,173289,111,Fisk Rd,138909730
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [67]:
# The man got into a car with a plate that included "H42W"
query= 'SELECT * FROM drivers_license WHERE id = "423327"'

ejecutar_query(query=query)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS


## Solución

Según mi investigación he llegado a la conclusión de que el asesino fue JEREMY BOWERS y al verificar el resultado en la web me dice lo siguiente:
"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."

In [68]:
query= 'SELECT * FROM interview WHERE person_id = "67318"'

ejecutar_query(query=query)

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


In [70]:
query= "SELECT * FROM facebook_event_checkin WHERE date LIKE '201712%'"

ejecutar_query(query=query)

,person_id,event_id,event_name,date
0,31811,8542,-- Herbert Prochnow\n,20171203
1,41421,3799,"And everywhere this language went,\n",20171206
2,66493,816,Just because your doctor has a name for your condition doesn't mean he\n,20171211
3,96520,8784,Good advice is something a man gives when he is too old to set a bad\n,20171218
4,33355,1973,He sure is a fun god\n,20171205
...,...,...,...,...
1272,24556,1143,SQL Symphony Concert,20171224
1273,99716,1143,SQL Symphony Concert,20171206
1274,99716,1143,SQL Symphony Concert,20171212
1275,99716,1143,SQL Symphony Concert,20171229


In [71]:
query= "SELECT * FROM person WHERE id = '99716' "

ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [72]:
query= "SELECT * FROM drivers_license WHERE id = '202298'"

ejecutar_query(query=query)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S


## Conclusión final

El asesino fue Jeremy Bowers pero este fue contratado por MIRANDA PRIESTLY 
"Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!"